In [1]:
import PyPDF2
from tqdm import tqdm
from rank_bm25 import BM25Okapi
import string
import torch

# Function to extract text from a PDF file using PyPDF2
def extract_text_from_pdf(pdf_path):
    corpus = []
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text = page.extract_text()
            if text:  # Ensure there's text on the page
                corpus.append({"passage": text.strip()})
    return corpus

# Function to split text into tokens
def split_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.lower().split()
    words = [word for word in words if len(word.strip()) > 0]
    return words

# Path to your PDF file
pdf_path = "/home/jupyter-trunglph/Others/RAG/21120157_21120415.pdf"  # Replace with your PDF path

# Extract the corpus from the PDF
meta_corpus = extract_text_from_pdf(pdf_path)
print("Meta corpus:", meta_corpus)

# Initiate BM25 retriever
tokenized_corpus = [split_text(doc["passage"]) for doc in tqdm(meta_corpus)]
print("Tokenized_corpus:", tokenized_corpus)
bm25 = BM25Okapi(tokenized_corpus)

Meta corpus: [{'passage': 'ĐẠI HỌC QUỐC GIA THÀNH PHỐ HỒ CHÍ MINH\nTRƯỜNG ĐẠI HỌC KHOA HỌC TỰ NHIÊN\nKHOA CÔNG NGHỆ THÔNG TIN\nTRÍ TUỆ NHÂN TẠO CHO AN NINH THÔNG TIN\nBÁO CÁO BÀI TẬP THỰC HÀNH\nLab 4 - Bảo mật trong Quá trình Huấn luyện\nMô hình Máy học\nThành viên:\n21120157 - Lê Phạm Hoàng Trung\n21120415 - Trần Ngọc Bảo\nKhoa: Công nghệ Thông tin\nThành phố Hồ Chí Minh - 2024'}, {'passage': 'Trường Đại học Khoa học Tự Nhiên TP.HCM Khoa: Công nghệ Thông tin\nMục lục\nI Thông tin chung 4\nII Nội dung báo cáo 5\n1 Mô hình gốc ban đầu 5\n1.1 Tổng quan về Federated Learning . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\n1.2 Cấu trúc mô hình sử dụng ResNet-18 trong FL . . . . . . . . . . . . . . . . . . . . . . 5\n1.3 Quy trình hoạt động của máy chủ (Server) . . . . . . . . . . . . . . . . . . . . . . . . 5\n1.4 Quy trình hoạt động của máy khách (Client) . . . . . . . . . . . . . . . . . . . . . . . 6\n1.5 Tổng hợp mô hình . . . . . . . . . . . . . . . . . . . . . . . . . . 

100%|██████████| 14/14 [00:00<00:00, 4443.12it/s]

Tokenized_corpus: [['đại', 'học', 'quốc', 'gia', 'thành', 'phố', 'hồ', 'chí', 'minh', 'trường', 'đại', 'học', 'khoa', 'học', 'tự', 'nhiên', 'khoa', 'công', 'nghệ', 'thông', 'tin', 'trí', 'tuệ', 'nhân', 'tạo', 'cho', 'an', 'ninh', 'thông', 'tin', 'báo', 'cáo', 'bài', 'tập', 'thực', 'hành', 'lab', '4', 'bảo', 'mật', 'trong', 'quá', 'trình', 'huấn', 'luyện', 'mô', 'hình', 'máy', 'học', 'thành', 'viên', '21120157', 'lê', 'phạm', 'hoàng', 'trung', '21120415', 'trần', 'ngọc', 'bảo', 'khoa', 'công', 'nghệ', 'thông', 'tin', 'thành', 'phố', 'hồ', 'chí', 'minh', '2024'], ['trường', 'đại', 'học', 'khoa', 'học', 'tự', 'nhiên', 'tphcm', 'khoa', 'công', 'nghệ', 'thông', 'tin', 'mục', 'lục', 'i', 'thông', 'tin', 'chung', '4', 'ii', 'nội', 'dung', 'báo', 'cáo', '5', '1', 'mô', 'hình', 'gốc', 'ban', 'đầu', '5', '11', 'tổng', 'quan', 'về', 'federated', 'learning', '5', '12', 'cấu', 'trúc', 'mô', 'hình', 'sử', 'dụng', 'resnet18', 'trong', 'fl', '5', '13', 'quy', 'trình', 'hoạt', 'động', 'của', 'máy', 'ch

In [2]:
def retrieve(question, topk=50):
    """
    Get most relevant chunks to the question using simple BM25 score.
    """
    tokenized_query = split_text(question)
    bm25_scores = bm25.get_scores(tokenized_query)
    corpus_size = len(meta_corpus)
    for i in range(corpus_size):
        meta_corpus[i]["bm25_score"] = bm25_scores[i]
    bm25_passages = sorted(meta_corpus, key=lambda x: x["bm25_score"], reverse=True)
    return bm25_passages[:topk]

In [3]:
question = "Cấu trúc mô hình sử dụng ResNet-18 trong Federated Learning?"
top_passages = retrieve(question, topk=3)
top_passages

[{'passage': 'Trường Đại học Khoa học Tự Nhiên TP.HCM Khoa: Công nghệ Thông tin\nMục lục\nI Thông tin chung 4\nII Nội dung báo cáo 5\n1 Mô hình gốc ban đầu 5\n1.1 Tổng quan về Federated Learning . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\n1.2 Cấu trúc mô hình sử dụng ResNet-18 trong FL . . . . . . . . . . . . . . . . . . . . . . 5\n1.3 Quy trình hoạt động của máy chủ (Server) . . . . . . . . . . . . . . . . . . . . . . . . 5\n1.4 Quy trình hoạt động của máy khách (Client) . . . . . . . . . . . . . . . . . . . . . . . 6\n1.5 Tổng hợp mô hình . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\nIII Nội dung báo cáo 7\n2 Xây dựng mô hình tấn công 7\n2.1 Các bước triển khai PGD Attack . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7\n2.2 Tích hợp vào client . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 8\n3 Xây dựng mô hình phòng thủ tại Server 8\n3.1 Tại sao lựa chọn Adversarial Training? . . . . . . . . . . . .

In [4]:
prompt_template = (
    "### System:\n"
    "Below is an instruction that describes a task, paired with an input that provides further context. "
    "Write a response that appropriately completes the request.\n\n\n\n"
    "### Instruction:\n{instruction}\n\n"
    "### Input:\n{input}\n\n"
    "### Response:\n{output}"
)

def get_prompt(question, contexts):
    context = "\n\n".join([f"Context [{i+1}]: {x['passage']}" for i, x in enumerate(contexts)])
    instruction = 'You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer.'
    input = f"Dựa vào một số ngữ cảnh được cho dưới đây, trả lời câu hỏi ở cuối.\n\n{context}\n\nQuestion: {question}\nHãy trả lời chi tiết và đầy đủ."
    prompt = prompt_template.format(
        instruction=instruction,
        input=input,
        output=''
    )
    return prompt

In [5]:
## Let's see how a prompt fed to the LLM looks like
contexts = retrieve(question, topk=3)
prompt = get_prompt(question, contexts)
print(prompt)

### System:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.



### Instruction:
You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer.

### Input:
Dựa vào một số ngữ cảnh được cho dưới đây, trả lời câu hỏi ở cuối.

Context [1]: Trường Đại học Khoa học Tự Nhiên TP.HCM Khoa: Công nghệ Thông tin
Mục lục
I Thông tin chung 4
II Nội dung báo cáo 5
1 Mô hình gốc ban đầu 5
1.1 Tổng quan về Federated Learning . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5
1.2 Cấu trúc mô hình sử dụng ResNet-18 trong FL . . . . . . . . . . . . . . . . . . . . . . 5
1.3 Quy trình hoạt động của máy chủ (Server) . . . . . . . . . . . . . . . . . . . . . . . . 5
1.4 Quy trình hoạt động của máy khách (Client) . . . . . . . . . . . . . . . . . . . . . . . 6
1.5 Tổng hợp mô hình . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [6]:
from transformers import GenerationConfig, TextStreamer
from transformers import LlamaForCausalLM, LlamaTokenizer, LlamaConfig
import torch

# torch_dtype = torch.bfloat16
model_id = "llm4fun/vietrag-7b-v1.0"
device = "cuda"

tokenizer = LlamaTokenizer.from_pretrained(model_id)
# model = LlamaForCausalLM.from_pretrained(
#     model_id,
#     config=LlamaConfig.from_pretrained(model_id),
#     torch_dtype=torch_dtype
# )

from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True  # Use 8-bit precision to reduce memory usage
)

model = LlamaForCausalLM.from_pretrained(
    model_id,
    config=LlamaConfig.from_pretrained(model_id),
    torch_dtype=torch.bfloat16,
    device_map="auto",  # Let Hugging Face automatically allocate the model
    quantization_config=quantization_config
)


model = model.eval().to(device)

/home/jupyter-trunglph/.conda/envs/RAG/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:32<00:00, 16.01s/it]


In [7]:
def generate(prompt, max_new_tokens=1024):
    """Text completion with a given prompt. In other words, give answer to your question.
    Args:
        prompt (str): Basically <instruction> + <question> + <retrieved_context>
        generation_config (not existed yet): For now, please manually tweak hyperparameters
        for generation in the `generation_config` below. Uncomment necessary arguments as you wish.
    Returns:
        list: an answer to the question within the prompt.
    """
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    model.eval()
    with torch.no_grad():
        generation_config = GenerationConfig(
            repetition_penalty=1.13,
            max_new_tokens=max_new_tokens,
            # temperature=0.2,
            # top_p=0.95,
            # top_k=20,
            # bos_token_id=tokenizer.bos_token_id,
            # eos_token_id=tokenizer.eos_token_id,
            # eos_token_id=0, # for open-end generation.
            pad_token_id=tokenizer.pad_token_id,
            do_sample=False,
            use_cache=True,
            return_dict_in_generate=True,
            output_attentions=False,
            output_hidden_states=False,
            output_scores=False,
        )
        streamer = TextStreamer(tokenizer, skip_prompt=True)
        generated = model.generate(
            inputs=input_ids,
            generation_config=generation_config,
            streamer=streamer,
        )

    gen_tokens = generated["sequences"].cpu()[:, len(input_ids[0]):]
    output = tokenizer.batch_decode(gen_tokens)[0]
    output = output.split(tokenizer.eos_token)[0]
    return output.strip()

In [8]:
output = generate(prompt)

Cấu trúc mô hình sử dụng ResNet-18 trong Federated Learning bao gồm một mô hình toàn cục với đặc trưng được trích xuất bằng ResNet-18. Mô hình này được tạo ra bởi máy chủ (server), sau đó nhận dữ liệu từ các máy khách (client) để huấn luyện. Dữ liệu này sau đó được tổng hợp bởi máy chủ, đảm bảo rằng kiến thức từ các client được hợp nhất mà không cần chia sẻ dữ liệu. Sau đó, mô hình được sử dụng để phân loại dữ liệu kiểm tra.</s>


From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


In [9]:
def rag(question, topk=3):
    top_passages = retrieve(question, topk=topk)
    retrieved_context = "\n\n".join([f"Context [{i+1}]: {x['passage']}" 
                    for i, x in enumerate(top_passages)])
    prompt = get_prompt(question, contexts)
    output = generate(prompt)
    result = {
        "retrieved_context": retrieved_context,
        "generated_answer": output
    }
    return result

In [10]:
questions = [
    "Federated Learning là gì?",
    "Resnet18 là gì?",
    "Quy định hoạt động của máy chủ (Server)?",
]
for question in questions:
    print(f"Câu hỏi: {question}")
    output = rag(question)
    print("---" * 30)

Câu hỏi: Federated Learning là gì?
Federated Learning (FL) là một phương pháp học máy phân tán, nơi nhiều thiết bị hoặc nút tính toán cục bộ tham gia vào việc huấn luyện một mô hình chung mà không cần chia sẻ dữ liệu thô. Thay vào đó, các thiết bị này cập nhật mô hình cục bộ và gửi các tham số (weights) hoặc gradients đã học được về cho máy chủ trung tâm. Máy chủ thực hiện việc tổng hợp các tham số này để tạo ra một mô hình toàn cục được cải tiến, sau đó gửi mô hình cập nhật lại cho các thiết bị để tiếp tục huấn luyện [2].</s>
------------------------------------------------------------------------------------------
Câu hỏi: Resnet18 là gì?
ResNet18 là một kiến trúc mạng nơ-ron tích chập (CNN), được sử dụng trong bài toán này để trích xuất đặc trưng từ dữ liệu hình ảnh. Nó được sử dụng làm trình trích xuất đặc trưng trong mô hình LogisticRegressionModelV2, nơi ResNet-18 đóng vai trò nền tảng, kết hợp với lớp hồi quy logistic để hoàn thiện quy trình huấn luyện.</s>
---------------------